In [1]:
import pandas as pd

In [119]:
df_all = pd.read_csv('/Users/mr.tian/Desktop/flight_data_2024.csv')

/var/folders/_t/99rhfqy12md_q88dy260m5d80000gn/T/ipykernel_92293/3266533460.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv('/Users/mr.tian/Desktop/flight_data_2024.csv')


In [12]:
df_all.head()

,year,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_fl_num,origin,origin_city_name,origin_state_nm,...,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,1,1,1,2024-01-01,9E,4814.0,JFK,"New York, NY",New York,...,0,136.0,122.0,84.0,509.0,0,0,0,0,0
1,2024,1,1,1,2024-01-01,9E,4815.0,MSP,"Minneapolis, MN",Minnesota,...,0,130.0,114.0,88.0,622.0,0,0,0,0,0
2,2024,1,1,1,2024-01-01,9E,4817.0,JFK,"New York, NY",New York,...,0,106.0,90.0,61.0,288.0,0,0,0,0,0
3,2024,1,1,1,2024-01-01,9E,4817.0,RIC,"Richmond, VA",Virginia,...,0,111.0,76.0,51.0,288.0,0,0,0,0,0
4,2024,1,1,1,2024-01-01,9E,4818.0,DTW,"Detroit, MI",Michigan,...,0,79.0,70.0,45.0,237.0,0,0,0,0,0


In [15]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7079081 entries, 0 to 7079080
Data columns (total 35 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   month                int64  
 2   day_of_month         int64  
 3   day_of_week          int64  
 4   fl_date              object 
 5   op_unique_carrier    object 
 6   op_carrier_fl_num    float64
 7   origin               object 
 8   origin_city_name     object 
 9   origin_state_nm      object 
 10  dest                 object 
 11  dest_city_name       object 
 12  dest_state_nm        object 
 13  crs_dep_time         int64  
 14  dep_time             float64
 15  dep_delay            float64
 16  taxi_out             float64
 17  wheels_off           float64
 18  wheels_on            float64
 19  taxi_in              float64
 20  crs_arr_time         int64  
 21  arr_time             float64
 22  arr_delay            float64
 23  cancelled            int64  
 24

In [17]:
df_all.isna().sum()

year                         0
month                        0
day_of_month                 0
day_of_week                  0
fl_date                      0
op_unique_carrier            0
op_carrier_fl_num            1
origin                       0
origin_city_name             0
origin_state_nm              0
dest                         0
dest_city_name               0
dest_state_nm                0
crs_dep_time                 0
dep_time                 92659
dep_delay                92970
taxi_out                 95734
wheels_off               95734
wheels_on                97856
taxi_in                  97856
crs_arr_time                 0
arr_time                 97854
arr_delay               113814
cancelled                    0
cancellation_code      6982766
diverted                     0
crs_elapsed_time             1
actual_elapsed_time     113814
air_time                113814
distance                     0
carrier_delay                0
weather_delay                0
nas_dela

### Keep only the features required for model training


In [121]:
keep_cols = [
    "year","month","day_of_week","fl_date",
    "op_unique_carrier","op_carrier_fl_num",
    "origin","dest",
    "crs_dep_time","crs_arr_time",
    "distance",
    "cancelled","diverted",
    "arr_delay"
]
df = df_all[keep_cols]

### Filter out cancelled and diverted flights


In [123]:
df = df[(df['cancelled'] == 0) & (df['diverted'] == 0)]
df = df.dropna(subset=['arr_delay'])

### Change dtype

In [125]:
for c in ['op_unique_carrier', 'origin', 'dest']:
    df[c] = df[c].astype('category')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6965267 entries, 0 to 7079080
Data columns (total 14 columns):
 #   Column             Dtype   
---  ------             -----   
 0   year               int16   
 1   month              int8    
 2   day_of_week        int8    
 3   fl_date            object  
 4   op_unique_carrier  category
 5   op_carrier_fl_num  float64 
 6   origin             category
 7   dest               category
 8   crs_dep_time       int64   
 9   crs_arr_time       int64   
 10  distance           float32 
 11  cancelled          int64   
 12  diverted           int64   
 13  arr_delay          float32 
dtypes: category(3), float32(2), float64(1), int16(1), int64(4), int8(2), object(1)
memory usage: 484.9+ MB


In [127]:
df["year"] = df["year"].astype("int16")
df["month"] = df["month"].astype("int8")
df["day_of_week"] = df["day_of_week"].astype("int8")

In [129]:
df["distance"] = df["distance"].astype("float32")
df["arr_delay"] = df["arr_delay"].astype("float32")

In [131]:
df['crs_dep_time'].head()

0    1252
1    1015
2    1415
3    1650
4    1015
Name: crs_dep_time, dtype: int64

### Convert scheduled time into interpretable hour and minute features

In [134]:
def hhmm_to_hour_min(s):
    s = s.fillna(0).astype(int)
    hour = s // 100
    minute = s % 100
    return hour.astype('int8'), minute.astype('int8')

df['dep_hour'], df['dep_min'] = hhmm_to_hour_min(df['crs_dep_time'])
df['arr_hour'], df['arr_min'] = hhmm_to_hour_min(df['crs_arr_time'])

In [136]:
df.drop(columns=['crs_dep_time', 'crs_arr_time'], inplace=True)

In [138]:
df.describe()

,year,month,day_of_week,op_carrier_fl_num,distance,cancelled,diverted,arr_delay,dep_hour,dep_min,arr_hour,arr_min
count,6965267.0,6.965267e+06,6.965267e+06,6.965266e+06,6.965267e+06,6965267.0,6965267.0,6.965267e+06,6.965267e+06,6.965267e+06,6.965267e+06,6.965267e+06
mean,2024.0,6.597649e+00,3.981578e+00,2.503915e+03,8.336735e+02,0.0,0.0,7.098246e+00,1.299242e+01,2.690934e+01,1.460619e+01,2.970598e+01
std,0.0,3.397294e+00,2.012222e+00,1.651642e+03,5.926239e+02,0.0,0.0,5.715171e+01,4.897566e+00,1.820891e+01,5.183259e+00,1.774730e+01
min,2024.0,1.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01,0.0,0.0,-1.260000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2024.0,4.000000e+00,2.000000e+00,1.151000e+03,3.990000e+02,0.0,0.0,-1.500000e+01,9.000000e+00,1.000000e+01,1.100000e+01,1.500000e+01
50%,2024.0,7.000000e+00,4.000000e+00,2.227000e+03,6.800000e+02,0.0,0.0,-6.000000e+00,1.300000e+01,2.700000e+01,1.500000e+01,3.000000e+01
75%,2024.0,1.000000e+01,6.000000e+00,3.717000e+03,1.069000e+03,0.0,0.0,9.000000e+00,1.700000e+01,4.300000e+01,1.900000e+01,4.500000e+01
max,2024.0,1.200000e+01,7.000000e+00,8.819000e+03,5.095000e+03,0.0,0.0,3.803000e+03,2.400000e+01,5.900000e+01,2.300000e+01,5.900000e+01


### Create discrete bins for delay minutes

In [141]:
df["arr_delay"] = df["arr_delay"].clip(lower=-60, upper=600)

In [143]:
df.head()

,year,month,day_of_week,fl_date,op_unique_carrier,op_carrier_fl_num,origin,dest,distance,cancelled,diverted,arr_delay,dep_hour,dep_min,arr_hour,arr_min
0,2024,1,1,2024-01-01,9E,4814.0,JFK,DTW,509.0,0,0,-19.0,12,52,15,8
1,2024,1,1,2024-01-01,9E,4815.0,MSP,CLE,622.0,0,0,-30.0,10,15,13,25
2,2024,1,1,2024-01-01,9E,4817.0,JFK,RIC,288.0,0,0,-20.0,14,15,16,1
3,2024,1,1,2024-01-01,9E,4817.0,RIC,JFK,288.0,0,0,-42.0,16,50,18,41
4,2024,1,1,2024-01-01,9E,4818.0,DTW,MKE,237.0,0,0,-14.0,10,15,10,34


In [145]:
df["arr_delay"].describe(percentiles=[0.01,0.05,0.5,0.95,0.99])

count    6.965267e+06
mean     6.620877e+00
std      4.964169e+01
min     -6.000000e+01
1%      -3.700000e+01
5%      -2.700000e+01
50%     -6.000000e+00
95%      8.300000e+01
99%      2.150000e+02
max      6.000000e+02
Name: arr_delay, dtype: float64

In [147]:
df = df[
    df["dep_hour"].between(0, 23) &
    df["dep_min"].between(0, 59)
]


In [149]:
assert df["dep_hour"].between(0, 23).all()
assert df["dep_min"].between(0, 59).all()

In [151]:
def group_rare(series, min_count=1000):
    if series.dtype.name == "category":
        series = series.cat.add_categories(["OTHER"])

    vc = series.value_counts()
    rare = vc[vc < min_count].index
    return series.where(~series.isin(rare), "OTHER")

df["origin"] = group_rare(df["origin"], min_count=1000)
df["dest"]   = group_rare(df["dest"], min_count=1000)

In [153]:
df.to_parquet("/Users/mr.tian/Desktop/flights_2024_clean.parquet", index=False)

In [154]:
df = df.drop(columns=["op_carrier_fl_num"])

In [157]:
df = df.drop(columns=["fl_date"])

### Time split + train model

In [160]:
train = df[df['month'] <= 9]
test = df[df['month'] >= 10]

X_train = train.drop(columns=['arr_delay'])
y_train = train['arr_delay']

X_test = test.drop(columns=['arr_delay'])
y_test = test['arr_delay']

### Ridge

In [162]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [164]:
cat_cols = ["op_unique_carrier", "origin", "dest"]
num_cols = [c for c in X_train.columns if c not in cat_cols]

In [166]:
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

In [168]:
model = Pipeline([
    ("prep", preprocess),
    ("ridge", Ridge(alpha=1.0))
])

In [170]:
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [171]:
print("MAE:", mean_absolute_error(y_test, pred))

MAE: 26.34133336963558


### lightGBM

In [186]:
from lightgbm import LGBMRegressor

model_lgb = LGBMRegressor(
    objective="regression",
    n_estimators=500,   
    learning_rate=0.05,    
    num_leaves=63,   
    max_depth=-1,            
    min_child_samples=100,   
    subsample=0.8,           
    colsample_bytree=0.8,    
    reg_alpha=0.0,
    reg_lambda=1.0,
    n_jobs=-1,
    random_state=42
)
model_lgb.fit(X_train, y_train)
pred = model_lgb.predict(X_test)

print("MAE:", mean_absolute_error(y_test, pred))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021832 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 926
[LightGBM] [Info] Number of data points in the train set: 5199987, number of used features: 10
[LightGBM] [Info] Start training from score 8.367733
MAE: 21.058961776589197


### XGBoost

In [189]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    min_child_weight=100,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist", 
    n_jobs=-1,
    random_state=42
)

model_xgb.fit(X_train, y_train)
pred = model_xgb.predict(X_test)

print("MAE:", mean_absolute_error(y_test, pred))

MAE: 21.089439392089844


### CatBoost

In [196]:
from catboost import CatBoostRegressor

cat_features = [
    X_train.columns.get_loc(c)
    for c in ["op_unique_carrier", "origin", "dest"]
]
model_cat = CatBoostRegressor(
    loss_function="MAE",
    iterations=500,
    learning_rate=0.05,
    depth=8,
    random_seed=42,
    verbose=100
)

model_cat.fit(
    X_train, y_train,
    cat_features=cat_features
)

pred = model_cat.predict(X_test)
print("MAE:", mean_absolute_error(y_test, pred))

0:	learn: 24.5587835	total: 1.18s	remaining: 9m 47s
100:	learn: 23.6509424	total: 1m 26s	remaining: 5m 42s
200:	learn: 23.5456807	total: 2m 56s	remaining: 4m 22s
300:	learn: 23.4773347	total: 4m 24s	remaining: 2m 54s
400:	learn: 23.4234914	total: 5m 47s	remaining: 1m 25s
499:	learn: 23.3806164	total: 7m 8s	remaining: 0us
MAE: 19.21435245179417


In [198]:
model_cat.save_model(
    "flight_delay_catboost.cbm"
)